# Building the Agent to integrate with the Chat Interface on the face

## Create a new assistant with file_search enabled in the tools parameter of the Assistant

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='OPEN_API_KEY')

assistant = client.beta.assistants.create(
  name="Course Selection Assistant",
  description="An AI assistant to help students select courses based on their degree and career goals.",
  instructions="You are an AI assistant that helps students in selecting a course based on their Degree, degree requirement and future goal (e.g. Data Scientist, UX Designer, etc.). Student need to share their Student ID, Name, their degree program and then their career goals. Then based on files that you have access to you'll respond to the courses the student can take. Keep the conversation tone helpful, engaging, and strictly do not answer any other question that is not related to course curriculum and selection. Give your response in unformatted text and do not cite resources in anyway. I just want you to simply state your answer.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

## Upload files and add them to a Vector Store

In [ ]:
# Create a vector store to hold the files that will be used to answer questions.
# Create a vector store caled "Knowledge Base"
vector_store = client.vector_stores.create(name="Knowledge Base")

# Ready the files for upload to OpenAI
file_paths = [
    "Knowledge Base/2023-2024 Handbook for the Master of Data Science Degree.pdf",
    "Knowledge Base/BS ComputerScience Degree Handbook.pdf",
    "Knowledge Base/BA Computer Science Degree Handbook.pdf",
    "Knowledge Base/BS Cybersecurity & Global Policy Degree Handbook.pdf",
    "Knowledge Base/BS Data Science Degree Handbook.pdf",
    "Knowledge Base/BS Informatics Degree Handbook.pdf",
    "Knowledge Base/BS Intelligent Systems Engineering Degree Handbook.pdf",
    "Knowledge Base/Computer Science Graduate Degree Handbook.pdf",
    "Knowledge Base/Doctor of Philosophy in Informatics 2020 Guidebook.pdf",
    "Knowledge Base/HCI-d Graduate Programs 2022-23.pdf",
    "Knowledge Base/Informatics Department Master of Science Degrees 2020 Guidebook.pdf",
    "Knowledge Base/Intelligent Systems Engineering Graduate Degree Handbook.pdf",
    "Knowledge Base/Master’s Student Handbook for MLS, MIS 4 plus 1 Degrees.pdf"
]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# Print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


## Update the assistant to use the new Vector Store

In [ ]:
# Update the assistant to use the vector store
# and the file search tool.
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

### Agent Building Completed!